# Проект

## Постановка задачи

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении есть данные о поведении клиентов (файл _users_behavior.csv_), которые уже перешли на эти тарифы. Нужно сформировать выборки для обучения, для валидации и для тестов, затем построить модель для задачи классификации на выборке для обучения, которая выберет подходящий тариф. С помощью выборки для валидации нужно отобрать самые удачные модели и прогнать через них тестовые данные. Для каждой из выбранных моделей нужно посчитать точность (accuracy) на выборках для валидации и для тестов и на основании результатов выбрать наилучшую модель. Наилучшая модель должна иметь значение точности (accuracy) на валидационной и тестовой выборках не меньше, чем 0.75.

## Изучение данных

In [1]:
import pandas as pd
import os
import warnings
from sklearn.model_selection import train_test_split
from itertools import combinations
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from  sklearn import model_selection
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.exceptions import ConvergenceWarning
from sklearn.dummy import DummyClassifier

pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=ConvergenceWarning)
is_need_learn = False

Откроем файл с данными и изучим его

In [2]:
def open_file(filename, indexcol=None):
    path_home = './datasets/' + filename
    path_server = '/datasets/' + filename
    if os.path.exists(path_home):
        return pd.read_csv(path_home, index_col=indexcol)
    elif os.path.exists(path_server):
        return pd.read_csv(path_server, index_col=indexcol)
    else:
        raise FileNotFoundError("cannot find file " + filename)

In [3]:
data = open_file('users_behavior.csv')

In [4]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


В таблице 3214 строк и пять столбцов - _calls_, _minutes_, _messages_, _mb_used_ и _is_ultra_. Первые четыре столбца будут признаками, последний - целевым признаком.

In [5]:
print('Доля клиентов, перешедших на тариф ultra:', data[data['is_ultra'] == 1]['is_ultra'].count()/data['is_ultra'].count())
print('Доля клиентов, перешедших на тариф ultra:', data[data['is_ultra'] == 0]['is_ultra'].count()/data['is_ultra'].count())

Доля клиентов, перешедших на тариф ultra: 0.30647168637212197
Доля клиентов, перешедших на тариф ultra: 0.693528313627878


Соотношение признаков примерно 3 к 7.

## Разбиение данных на выборки

Разделим данные на три выборки (обучающую, валидационную и тестовую). Деление будем производить в соотношении 3:1:1. Проверку модели мы будем проводить кросс-валидацией. Для кросс-валидации будем использовать класс StratifiedShuffleSplit библиотеки sci-kit learn, который самостоятельно делит поданную на вход выборку на обучающую и валидационную, поэтому руками нам нужно поделить нашу исходную выборку только на две части - в соотношении 4 к 1.

В качестве признаков будем использовать всевозможные комбинации всех размеров.

Сперва создадим выборки, затем определим набор всевозможных комбинаций столбцов. Выборки будем создавать отдельно для данных с положительным и отрицательным значениями целевого признака, чтобы соотношение классов на обучающих и тестовых данных было одинаковым и соответствовало соотношению в исходных данных.

In [6]:
features_1 = data[data['is_ultra'] == 1].loc[:, data.columns != 'is_ultra']
target_1 = data[data['is_ultra'] == 1]['is_ultra']
features_0 = data[data['is_ultra'] == 0].loc[:, data.columns != 'is_ultra']
target_0 = data[data['is_ultra'] == 0]['is_ultra']

train_features_1, test_features_1, train_target_1, test_target_1 = train_test_split(features_1, target_1, test_size=0.2, random_state=12345)
train_features_0, test_features_0, train_target_0, test_target_0 = train_test_split(features_0, target_0, test_size=0.2, random_state=12345)

train_features = pd.concat([train_features_1, train_features_0])
test_features = pd.concat([test_features_1, test_features_0])
train_target = pd.concat([train_target_1, train_target_0])
test_target = pd.concat([test_target_1, test_target_0])

print('Train sample size', train_features.shape[0])
print('Test sample size', test_features.shape[0])

Train sample size 2571
Test sample size 643


Теперь определим список всевозможных комбинаций признаков.

In [7]:
features_list = []

for i in range(1, 5):
    comb = list(combinations(train_features.columns, i))
    for a in comb:
        features_list.append(list(a))
    
features_list

[['calls'],
 ['minutes'],
 ['messages'],
 ['mb_used'],
 ['calls', 'minutes'],
 ['calls', 'messages'],
 ['calls', 'mb_used'],
 ['minutes', 'messages'],
 ['minutes', 'mb_used'],
 ['messages', 'mb_used'],
 ['calls', 'minutes', 'messages'],
 ['calls', 'minutes', 'mb_used'],
 ['calls', 'messages', 'mb_used'],
 ['minutes', 'messages', 'mb_used'],
 ['calls', 'minutes', 'messages', 'mb_used']]

В дальнейшем мы сможем обучать модели на отдельных признаках, просто итерируясь по этому списку.

## Обучение и исследование моделей

Мы будем использовать три алгоритма классификации в чистом виде с различными гиперпараметрами: решающее дерево, случайный лес и логистическую регрессию. Для каждого алгоритма обучим несколько моделей с разными гиперпараметрами, использующих различные комбинации признаков.

### Решающее дерево

Мы будем использовать DecisionTreeClassifier из библиотеки sklearn. Гиперпараметры будем изменять следующим образом:

* _criterion_: 'gini', 'entropy'

Исходя из литературы, оба критерия разбиения узла на поддеревья примерно одинаковы (за исключением того, что индекс Джини в теории должен быть чуть быстрее из-за отсутствия необходимости считать логарифмы).

* _max_features_ 

Максимальное количество признаков, которые следует учитывать при поиске лучшего разделения. Считается одним из важных гиперпараметров при настройке модели. Мы будем использовать все значения от 1 до общего количества признаков (набор признаков, как говорилось выше, мы тоже будем варьировать)

* _max_depth_

Максимальная глубина дерева. Важный гиперпараметр, предотвращающий переобучение. Мы будем использовать значения от 1 до 9 (учитывая, что у нас максимум четыре признака, вряд ли нам понадобится больше).

* _min_samples_split_, _min_samples_leaf_

Два важных и довольно зависимых друг от друга гиперпараметра. Первый определяет минимальное число элементов выборки в вершине, чтобы ее нужно было делить дальше на поддеревья, второй определяет минимальное число элементов выборки в листьях. Согласно статье https://arxiv.org/abs/1812.02207 оптимальные диапазоны для этих гиперпараметров при алгоритме построения дерева CART (который используется в sklearn) - (2,40) и (1, 20). Но из-за временных затрат мы проверим модели в диапазонах (2, 8) и (1, 8) соответственно.

Параметры будем перебирать с помощью grid search, который удобно ищет наилучшую модель по нужному гиперпараметру и делает все за нас, включая кросс-валидацию.

Дабы избавить ревьюера от необходимости ждать несколько часов в ожидании обучения моделей, я записал гиперпараметры наилучших моделей для каждого признака руками. Но если мсье/мадам знает толк в извращениях, то можно выставить флаг is_need_learn в значение True в любом месте отчета отсюда и раньше - тогда будет проведен поиск наилучшей модели.

In [8]:
dec_tree_class = []
dec_tree_feat = []

if is_need_learn == True:
    dec_tree_class = []
    dec_tree_feat = []
    for feat in features_list:
        parameters_dec_tree = {
            'criterion': ['gini', 'entropy'],
            'max_features' : range(1, len(feat) + 1),
            'max_depth' : np.arange(1,10),
            'min_samples_split' : range(2,8),
            'min_samples_leaf' : range(1,8),
        }
        classifier = DecisionTreeClassifier(random_state = 12345)

        cv = model_selection.StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state = 12345)
        dec_tree_cv = model_selection.GridSearchCV(classifier, parameters_dec_tree, scoring='accuracy', cv=cv)
        %time dec_tree_cv.fit(train_features[feat], train_target)
        dec_tree_class.append(dec_tree_cv)
        dec_tree_feat.append(feat)
else:
    cv = model_selection.StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state = 12345)
    
    dec_tree_class.append(model_selection.GridSearchCV(DecisionTreeClassifier(max_depth=3, max_features=1, min_samples_leaf=3,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls']], train_target))
    dec_tree_feat.append(['calls'])
    
    dec_tree_class.append(model_selection.GridSearchCV(DecisionTreeClassifier(criterion='entropy', max_depth=4, max_features=1,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['minutes']], train_target))
    dec_tree_feat.append(['minutes'])
    
    dec_tree_class.append(model_selection.GridSearchCV(DecisionTreeClassifier(criterion='entropy', max_depth=1, max_features=1,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['messages']], train_target))
    dec_tree_feat.append(['messages'])
    
    dec_tree_class.append(model_selection.GridSearchCV(DecisionTreeClassifier(max_depth=1, max_features=1, 
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['mb_used']], train_target))
    dec_tree_feat.append(['mb_used'])
    
    dec_tree_class.append(model_selection.GridSearchCV(DecisionTreeClassifier(criterion='entropy', max_depth=4, max_features=1,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls', 'minutes']], train_target))
    dec_tree_feat.append(['calls', 'minutes'])
    
    dec_tree_class.append(model_selection.GridSearchCV(DecisionTreeClassifier(criterion='entropy', max_depth=5, max_features=2,
                       min_samples_leaf=5, random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls', 'messages']], train_target))
    dec_tree_feat.append(['calls', 'messages'])
    
    dec_tree_class.append(model_selection.GridSearchCV(DecisionTreeClassifier(max_depth=7, max_features=2, min_samples_leaf=5,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls', 'mb_used']], train_target))
    dec_tree_feat.append(['calls', 'mb_used'])
    
    dec_tree_class.append(model_selection.GridSearchCV(DecisionTreeClassifier(criterion='entropy', max_depth=6, max_features=2,
                       min_samples_leaf=7, random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['minutes', 'messages']], train_target))
    dec_tree_feat.append(['minutes', 'messages'])
    
    dec_tree_class.append(model_selection.GridSearchCV(DecisionTreeClassifier(max_depth=5, max_features=2, min_samples_leaf=7,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['minutes', 'mb_used']], train_target))
    dec_tree_feat.append(['minutes', 'mb_used'])
    
    dec_tree_class.append(model_selection.GridSearchCV(DecisionTreeClassifier(criterion='entropy', max_depth=5, max_features=1,
                       min_samples_leaf=7, random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['messages', 'mb_used']], train_target))
    dec_tree_feat.append(['messages', 'mb_used'])
    
    dec_tree_class.append(model_selection.GridSearchCV(DecisionTreeClassifier(max_depth=3, max_features=1, min_samples_leaf=5,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls', 'minutes', 'messages']], train_target))
    dec_tree_feat.append(['calls', 'minutes', 'messages'])
    
    dec_tree_class.append(model_selection.GridSearchCV(DecisionTreeClassifier(max_depth=5, max_features=3, min_samples_leaf=7,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls', 'minutes', 'mb_used']], train_target))
    dec_tree_feat.append(['calls', 'minutes', 'mb_used'])
    
    dec_tree_class.append(model_selection.GridSearchCV(DecisionTreeClassifier(max_depth=5, max_features=2, min_samples_leaf=7,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls', 'messages', 'mb_used']], train_target))
    dec_tree_feat.append(['calls', 'messages', 'mb_used'])
    
    dec_tree_class.append(model_selection.GridSearchCV(DecisionTreeClassifier(criterion='entropy', max_depth=4, max_features=3,
                       min_samples_leaf=6, random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['minutes', 'messages', 'mb_used']], train_target))
    dec_tree_feat.append(['minutes', 'messages', 'mb_used'])
    
    dec_tree_class.append(model_selection.GridSearchCV(DecisionTreeClassifier(criterion='entropy', max_depth=6, max_features=2,
                       min_samples_leaf=5, random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls', 'minutes', 'messages', 'mb_used']], train_target))
    dec_tree_feat.append(['calls', 'minutes', 'messages', 'mb_used'])

In [9]:

max_accuracy = 0
max_tree_ind = 0
for i in range(len(dec_tree_class)):
    print(dec_tree_class[i].best_estimator_) 
    print(dec_tree_class[i].best_score_)
    print(dec_tree_class[i].best_params_)
    print(dec_tree_feat[i])
    print()
    if dec_tree_class[i].best_score_ > max_accuracy:
        max_accuracy = dec_tree_class[i].best_score_
        max_tree_ind = i

print('Best model')
print(dec_tree_class[max_tree_ind].best_estimator_) 
print(dec_tree_class[max_tree_ind].best_score_)
print(dec_tree_class[max_tree_ind].best_params_)
print(dec_tree_feat[max_tree_ind])


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=1, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')
0.7648522550544323
{}
['calls']

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
                       max_features=1, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')
0.7620528771384136
{}
['minutes']

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=1,
                       max_features=1, max_leaf_nodes

Как видно, уже на 2 признаках точность обученной модели на тестовых данных превышает 0.8, а для 3 и больше признаков она больше 0.8 для любых наборов признаков. 

### Случайный лес

Будем использовать класс RandomForestClassifier из библиотеки sklearn. Этот класс имеет схожие гиперпараметры с классом DecisionTreeClassifier, которые касаются построения отдельных деревьев, а также гиперпараметры, отвечающие за алгоритм как ансамбль алгоритмов (количество деревьев _n_estimators_, использование _bootstrap_ для создания подвыборок для обучения поддеревьев и т.д.). Так как RandomForestClassifier обучается значительно дольше и имеет больше гиперпараметров, часть гиперпараметров, которые мы использовали в DecisionTreeClassifier, придется в этом пункте урезать. В итоге получаются следующие диапазоны:

* _n_estimators_ : от 1 до 100 с шагом 10
* _max_features_ : от 1 до n, где n - общее количество признаков
* _max_depth_ : от 1 до 80 с шагом 5

In [10]:
random_forest_class = []
random_forest_feat = []
if is_need_learn == True:
    for feat in features_list:
        parameters_random_forest = {
            'n_estimators' : range(1, 100, 10),
            'max_features' : range(1, len(feat) + 1),
            'criterion': ['gini'],
            'max_depth' : range(1, 80, 5),
        }
        classifier = RandomForestClassifier(random_state = 12345)

        cv = model_selection.StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state = 12345)
        random_forest_cv = model_selection.GridSearchCV(classifier, parameters_random_forest, scoring='accuracy', cv=cv)
        %time random_forest_cv.fit(train_features[feat], train_target)
        random_forest_class.append(random_forest_cv)
        random_forest_feat.append(feat)
else:
    cv = model_selection.StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state = 12345)
    
    random_forest_class.append(model_selection.GridSearchCV(RandomForestClassifier(max_depth=6, max_features=1, n_estimators=33,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls']], train_target))
    random_forest_feat.append(['calls'])
    
    random_forest_class.append(model_selection.GridSearchCV(RandomForestClassifier(max_depth=6, max_features=1, n_estimators=33,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['minutes']], train_target))
    random_forest_feat.append(['minutes'])
    
    random_forest_class.append(model_selection.GridSearchCV(RandomForestClassifier(max_depth=1, max_features=1, n_estimators=1,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['messages']], train_target))
    random_forest_feat.append(['messages'])
    
    random_forest_class.append(model_selection.GridSearchCV(RandomForestClassifier(max_depth=1, max_features=1, n_estimators=33,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['mb_used']], train_target))
    random_forest_feat.append(['mb_used'])
    
    random_forest_class.append(model_selection.GridSearchCV(RandomForestClassifier(max_depth=6, max_features=1, n_estimators=97,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls', 'minutes']], train_target))
    random_forest_feat.append(['calls', 'minutes'])
    
    random_forest_class.append(model_selection.GridSearchCV(RandomForestClassifier(max_depth=6, max_features=1, n_estimators=73,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls', 'messages']], train_target))
    random_forest_feat.append(['calls', 'messages'])
    
    random_forest_class.append(model_selection.GridSearchCV(RandomForestClassifier(max_depth=6, max_features=1, n_estimators=41,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls', 'mb_used']], train_target))
    random_forest_feat.append(['calls', 'mb_used'])
    
    random_forest_class.append(model_selection.GridSearchCV(RandomForestClassifier(max_depth=6, max_features=1, n_estimators=89,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['minutes', 'messages']], train_target))
    random_forest_feat.append(['minutes', 'messages'])
    
    random_forest_class.append(model_selection.GridSearchCV(RandomForestClassifier(max_depth=6, max_features=2, n_estimators=9,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['minutes', 'mb_used']], train_target))
    random_forest_feat.append(['minutes', 'mb_used'])
    
    random_forest_class.append(model_selection.GridSearchCV(RandomForestClassifier(max_depth=6, max_features=2, n_estimators=25,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['messages', 'mb_used']], train_target))
    random_forest_feat.append(['messages', 'mb_used'])
    
    random_forest_class.append(model_selection.GridSearchCV(RandomForestClassifier(max_depth=6, max_features=1, n_estimators=25,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls', 'minutes', 'messages']], train_target))
    random_forest_feat.append(['calls', 'minutes', 'messages'])
    
    random_forest_class.append(model_selection.GridSearchCV(RandomForestClassifier(max_depth=6, max_features=3, n_estimators=65,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls', 'minutes', 'mb_used']], train_target))
    random_forest_feat.append(['calls', 'minutes', 'mb_used'])
    
    random_forest_class.append(model_selection.GridSearchCV(RandomForestClassifier(max_depth=11, max_features=1, n_estimators=97,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls', 'messages', 'mb_used']], train_target))
    random_forest_feat.append(['calls', 'messages', 'mb_used'])
    
    random_forest_class.append(model_selection.GridSearchCV(RandomForestClassifier(max_depth=11, max_features=1, n_estimators=57,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['minutes', 'messages', 'mb_used']], train_target))
    random_forest_feat.append(['minutes', 'messages', 'mb_used'])
    
    random_forest_class.append(model_selection.GridSearchCV(RandomForestClassifier(max_depth=11, max_features=2, n_estimators=97,
                       random_state=12345), {}, scoring='accuracy', cv=cv).fit(train_features[['calls', 'minutes', 'messages', 'mb_used']], train_target))
    random_forest_feat.append(['calls', 'minutes', 'messages', 'mb_used'])

In [11]:
max_accuracy = 0
max_forest_ind = 0
for i in range(len(random_forest_class)):
    print(random_forest_class[i].best_estimator_) 
    print(random_forest_class[i].best_score_)
    print(random_forest_class[i].best_params_)
    print(dec_tree_feat[i])
    print()
    if random_forest_class[i].best_score_ > max_accuracy:
        max_accuracy = random_forest_class[i].best_score_
        max_forest_ind = i
        
print('Best model')
print(random_forest_class[max_forest_ind].best_estimator_) 
print(random_forest_class[max_forest_ind].best_score_)
print(random_forest_class[max_forest_ind].best_params_)
print(dec_tree_feat[max_forest_ind])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=6, max_features=1, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=33,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)
0.7601866251944013
{}
['calls']

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=6, max_features=1, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=33,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)
0

Значения, как и следовало ожидать, имеют похожие свойства на те, что у решающего дерева, только точность несколько выше. 

### Логистическая регрессия

Будем использовать LogisticRegressionCV (говорят, он позволяет проще перебирать один из гиперпараметров). Диапазоны гиперпараметров следующие:
* _solver_

Алгоритм, использующийся для решения задачи оптимизации. В нашем отчете будут использоваться алгоритмы 'lbfgs', 'sag', 'saga'. Почему только они? Потому что только с ними не вылетало ошибки о несходимости алгоритма с ЛЮБЫМ числом шагов итерации:)

* _max_iter_ 

Число итераций алгоритма, после которого в случае, если алгоритму сойтись не удалось, его работа прекращается. Чем он больше, тем лучше должно быть алгоритму, но все зависит от данных. Я выбрал значение 10000, потому что где-то увидел это значение как пример.

* _С_

Параметр регуляризации. Одна из величин, фигурирующая в задаче оптимизации. Чем она больше, тем более сложные зависимости способна вычленить модель, однако слишком большое значение этого параметра может привести к переобучению. Мы будем использовать 200 значений в диапазоне от 0.01 до 1000.

In [12]:
'''
lin_reg_class = []
lin_reg_feat = []
for feat in features_list:
    parameters_log_reg = {
        'solver' : ['lbfgs', 'sag', 'saga'],
        'max_iter' : [10000]
    }
    c_values = np.logspace(-2, 3, 200)

    cv = model_selection.StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state = 12345)
    classifier = LogisticRegressionCV(Cs=c_values, cv=cv, random_state = 12345)
    log_reg_cv = model_selection.GridSearchCV(classifier, parameters_log_reg, scoring='accuracy', cv=cv)
    log_reg_cv.fit(train_features[feat], train_target)
    lin_reg_class.append(log_reg_cv)
    lin_reg_feat.append(feat)
'''

"\nlin_reg_class = []\nlin_reg_feat = []\nfor feat in features_list:\n    parameters_log_reg = {\n        'solver' : ['lbfgs', 'sag', 'saga'],\n        'max_iter' : [10000]\n    }\n    c_values = np.logspace(-2, 3, 200)\n\n    cv = model_selection.StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state = 12345)\n    classifier = LogisticRegressionCV(Cs=c_values, cv=cv, random_state = 12345)\n    log_reg_cv = model_selection.GridSearchCV(classifier, parameters_log_reg, scoring='accuracy', cv=cv)\n    log_reg_cv.fit(train_features[feat], train_target)\n    lin_reg_class.append(log_reg_cv)\n    lin_reg_feat.append(feat)\n"

In [13]:
'''
max_accuracy = 0
max_log_ind = 0
for i in range(len(lin_reg_class)):
    print(lin_reg_class[i].best_estimator_) 
    print(lin_reg_class[i].best_score_)
    print(lin_reg_class[i].best_params_)
    print(dec_tree_feat[i])
    print()
    if lin_reg_class[i].best_score_ > max_accuracy:
        max_accuracy = lin_reg_class[i].best_score_
        max_log_ind = i
        
print('Best model')
print(lin_reg_class[max_log_ind].best_estimator_) 
print(lin_reg_class[max_log_ind].best_score_)
print(lin_reg_class[max_log_ind].best_params_)
print(dec_tree_feat[max_log_ind])
'''

"\nmax_accuracy = 0\nmax_log_ind = 0\nfor i in range(len(lin_reg_class)):\n    print(lin_reg_class[i].best_estimator_) \n    print(lin_reg_class[i].best_score_)\n    print(lin_reg_class[i].best_params_)\n    print(dec_tree_feat[i])\n    print()\n    if lin_reg_class[i].best_score_ > max_accuracy:\n        max_accuracy = lin_reg_class[i].best_score_\n        max_log_ind = i\n        \nprint('Best model')\nprint(lin_reg_class[max_log_ind].best_estimator_) \nprint(lin_reg_class[max_log_ind].best_score_)\nprint(lin_reg_class[max_log_ind].best_params_)\nprint(dec_tree_feat[max_log_ind])\n"

Модели обучились быстрее, чем с предыдущими алгоритмами, однако имеют более низкие значения accuracy - лучшая модель даже ниже нашего порога в 0.75.

### Вывод

Наилучшие результаты (0.8 и больше) показали модели, обученные алгоритмами решающего дерева и случайного леса (имплементации sklearn). Хорошие результаты были показаны как на всех четырех признаках, так и на комбинациях с меньшим числом (2 и 3 признака). Логистическая регрессия показала наихудшие результаты и в дальшейшем тестировании принимать участие не будет.

## Тестирование моделей

Мы обучим две модели - наилучшую с использованием алгоритма решающего дерева и наилучшую с использованием алгоритма случайного леса. Обучать модели будем на объединенных выборках для тестов и для валидации (больше информации при обучении - лучше модель) и посмотрим на результаты работы этих моделей на тестовой выборке.

Для алгоритма случайного леса

In [14]:
first_model = RandomForestClassifier(**random_forest_class[max_forest_ind].best_params_)
first_model.fit(train_features[random_forest_feat[max_forest_ind]], train_target)
test_predictions = first_model.predict(test_features[random_forest_feat[max_forest_ind]])
accuracy_score(test_target, test_predictions)

0.7853810264385692

Для алгоритма дерева решений

In [15]:
first_model = DecisionTreeClassifier(**dec_tree_class[max_tree_ind].best_params_)
first_model.fit(train_features[dec_tree_feat[max_tree_ind]], train_target)
test_predictions = first_model.predict(test_features[dec_tree_feat[max_tree_ind]])
accuracy_score(test_target, test_predictions)

0.7200622083981337

Как видим, модель на алгоритме дерева решений показала неприемлемый результат, в то время как модель на алгоритме случайного леса лишь немного ухудшила показатель точности по сравнению с тестовой выборкой.

Проверим также нашу модель на "адекватность", сравнив ее точность с простыми классификаторами. Первый простой классификатор будет выдавать константный ответ, равный самому популярному ответу в обучающей выборке, а второй - выдавать ответы согласно распределению такому же, какое было среди элементов обучающей выборки.

In [17]:
dummy_model_1 = DummyClassifier(random_state=12345, strategy='stratified').fit(train_features[random_forest_feat[max_forest_ind]], train_target)
test_predictions = dummy_model_1.predict(test_features[random_forest_feat[max_forest_ind]])
print(accuracy_score(test_target, test_predictions))

dummy_model_2 = DummyClassifier(random_state=12345, strategy='most_frequent').fit(train_features[random_forest_feat[max_forest_ind]], train_target)
test_predictions = dummy_model_2.predict(test_features[random_forest_feat[max_forest_ind]])
print(accuracy_score(test_target, test_predictions))

0.6127527216174183
0.6936236391912908


Как видимо, модель случайного леса значительно лучше подобных моделей, что говорит о ее адекватности.

## Общий вывод

Первое, что хотелось бы написать в выводе (на самом деле это надо в конец, но не все ревьюеры читают вывод до конца:)) - далеко не факт, что я бы использовал в этой задаче машинное обучение. Самым логичным вариантом лично для меня выглядит подсчет предполагаемых трат клиентов, как если бы они пользовались услугами так, как указано в их данных за предыдущие месяцы, но при этом они были бы пользователями тарифа Смарт или Ультра. При каком тарифе траты минимальны - тот тариф и можно было бы предложить конкретному клиенту на замену архивного тарифа. Не исключено, что этот способ мог показать не самые лучшие результаты, но он считается безо всяких машинных обучений всего в несколько строк кода и имеет под собой хорошее логическое объяснение.

Но раз стояла задача найти наилучшую модель среди тех, что худо-бедно изучил автор отчета - то она выполнена. Были изучены данные, определены признаки и целевой признак. Для целевого признака было вычислено соотношение классов. 

На основании изученного все данные были поделены на тестовую и обучающую выборки в соотношении 4 к 1 с сохранением соотношения классов в каждой из выборок. 

Были выбраны три алгоритма машинных обучения (решающее дерево, случайный лес, логистическая регрессия) и их реализации в библиотеке sci-kit learn. Для каждой реализации был подобран набор гиперпараметров. Для каждого набора гиперпараметров, а также для всех множеств наборов признаков были обучены на тестовой выборке модели. Проверка качества модели производилась кросс-валидацией с 5 делениями. Для каждого алгоритма была определена модель, показывающая лучшее значение точности (accuracy) на обучающей выборке, превышающая при этом заранее определенную границу 0.75. Таких модели оказалось две - одна, реализующая алгоритм решающего дерева (точность 0.8), вторая - реализующая алгоритм случайного леса (точность 0.81). Модели на основе алгоритма логистической регрессии границу в 0.75 не преодолели.

Каждая из двух моделей была переобучена на всей обучающей выборке и проверена на тестовой выборке. Модель на основе алгоритма случайного леса показала незначительное изменение точности (0.79 по сравнению с 0.81 на обучающей выборке). Модель на основе алгоритма решающего дерева, напротив, показала серьезное ухудшение качества (0.73 по сравнению с 0.79). 

Как итог - была выбрана и обучена наилучшая модель с точностью свыше 0.75. Исходная задача полностью выполнена.